In [ ]:
import os
import pickle
import pandas as pd
from pickle import Pickler
import git
import subprocess
from os import getcwd
from os import chdir
from subprocess import Popen
from subprocess import PIPE, STDOUT
import shutil
import xml.etree.ElementTree as ET
import json
import tqdm.auto as tqdm

In [ ]:
dicts_c = os.listdir('dicts_c')
len(dicts_c)

In [ ]:
i = 54 #CHANGE IT FOR EVERY FILE

In [ ]:
file_name = dicts_c[i].split('.')[0]
repo_name = file_name.replace('_','/')
repo_name

In [ ]:
try:
    with open('dicts_c/'+dicts_c[i], 'rb') as f:
        file_content = pickle.load(f)
except MemoryError:
    print("File cannot be opened")

In [ ]:
file_content_df = pd.DataFrame.from_dict(file_content, orient='index')
hash_releases = file_content_df['release']
len(hash_releases)

In [ ]:
folder = repo_name.split('/')[-1]
repo = git.Repo.clone_from('https://github.com/'+repo_name+'.git', folder)#no_checkout=True)

In [ ]:
hash_and_dates = dict()

In [ ]:
ROOT_PATH = getcwd()

git.Repo(ROOT_PATH+'/'+folder)

# Change to directory
chdir(ROOT_PATH+'/'+folder)

for release in hash_releases:
    with Popen(
        args=['git', 'show', '-s', '--format=%ci' , release],
        shell=False,
        stdout=PIPE,
        bufsize=1,
        universal_newlines=True,
    ) as process:

        for line in process.stdout:
            hash_and_dates[release] = line.strip()

In [ ]:
chdir(ROOT_PATH)

In [ ]:
getcwd()

In [ ]:
oldest_release = sorted(hash_and_dates.values())[0]
oldest_release

In [ ]:
commit_for_checkout = list(hash_and_dates.keys())[list(hash_and_dates.values()).index(oldest_release)]
commit_for_checkout

In [ ]:
repo.git.checkout(commit_for_checkout)

In [ ]:
print("Execute nicad for each file for release "+commit_for_checkout)

p = Popen(['C:/cygwin64/bin/bash.exe', '-c', '. /etc/profile; '+
'nicad6cross functions c C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/to_check '+ 
           'C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/'+folder], 
  stdout=PIPE, stderr=STDOUT)
p.communicate()[0]

In [ ]:
idx = 0
files_to_check_with_cppcheck = dict()

if os.path.exists('to_check_functions-blind-crossclones/to_check_functions-blind-crossclones-0.30.xml'):
    tree = ET.parse('to_check_functions-blind-crossclones/to_check_functions-blind-crossclones-0.30.xml')
    root = tree.getroot()
    for clones in root.findall('clone'):
        for source in clones:
            if folder in source.attrib['file']:
                try:
                    files_to_check_with_cppcheck[idx] = {
                        'file': source.attrib['file'],
                        'start': source.attrib['startline'],
                        'end': source.attrib['endline']
                    }
                    idx = idx + 1
                except KeyError as e:
                    print(e)

In [ ]:
len(files_to_check_with_cppcheck.keys())

In [ ]:
chdir(ROOT_PATH+'/'+folder)

In [ ]:
getcwd()

In [ ]:
clone_snippets_with_vuln = dict()
idx = 0

key_n = 0
for key in files_to_check_with_cppcheck.keys():
    try:
        print("Key "+str(key_n))
        code_context = []
        path = files_to_check_with_cppcheck[key]['file']
        print(path)
        proc = subprocess.run(["cppcheck", "--enable=style", "--template={cwe};{file};{line};{severity};{code}", path], capture_output=True)

        print(proc.stdout)
        result = proc.stderr.decode('utf-8')

        if result:
            items = result.split('\r\n')
            items = list(filter(None, items))

            for x, y in zip(*[iter(items)] * 2):
                row = x + y
                row = row.split(';')
                print("Flaw found at line "+row[2])
                print("Clone start a line "+files_to_check_with_cppcheck[key]['start']+' and end at line '+files_to_check_with_cppcheck[key]['end'])
                if int(row[2])>=int(files_to_check_with_cppcheck[key]['start']) and int(row[2])<=int(files_to_check_with_cppcheck[key]['end']):
                    code_context = []
                    with open(path, 'rb') as f:
                        content = f.readlines()
                        start = int(files_to_check_with_cppcheck[key]['start']) - 1
                        end = int(files_to_check_with_cppcheck[key]['end']) - 1
                        for number in range(start, end+1):
                            code_context.append(content[number].decode('utf-8'))

                    #chdir(ROOT_PATH+'/'+folder)
                    with Popen(
                    args=['git', 'log', '--reverse', '--pretty=format:%H', '-L', 
                          files_to_check_with_cppcheck[key]['start']+','+files_to_check_with_cppcheck[key]['end']+":"+path],
                    shell=False,
                    stdout=PIPE,
                    bufsize=1,
                    universal_newlines=True,
                    ) as process:
                        for line in process.stdout:
                            #print("commit "+line.strip())
                            origin_commit = line.strip()
                            break;

                        with Popen(
                            args=['git', 'show', '-s', '--format=%ci' , origin_commit],
                            shell=False,
                            stdout=PIPE,
                            bufsize=1,
                            universal_newlines=True,
                        ) as process:

                            for line in process.stdout:
                                date = line.strip()

                    clone_snippets_with_vuln[idx]={'repo': repo_name, 'origin_commit': origin_commit, 'file': path, #path.split(folder)[1][1:], 
                                                   'code_context': ''.join(code_context), 'commit_date': date, 'line_vuln': row[2], 'code_vuln': row[4]}
                    idx = idx + 1
                print()
        key_n = key_n + 1
    except UnicodeDecodeError:
        print("Error")

In [ ]:
prefix = getcwd()
prefix = prefix.replace('\\', '/')
print(prefix)
for key in clone_snippets_with_vuln.keys():
    print(clone_snippets_with_vuln[key]['file'])
    clone_snippets_with_vuln[key]['file'] = clone_snippets_with_vuln[key]['file'].replace('C:/Users/Grazia/Desktop/Tesi/StackOverflow_Vulnerabilities/2-Vulnerabilty_Propagation/'+folder+'/', '')
    print(clone_snippets_with_vuln[key]['file'])

In [ ]:
 chdir(ROOT_PATH)

In [ ]:
getcwd()

In [ ]:
if clone_snippets_with_vuln:
    clone_snippets_with_vuln_df = pd.DataFrame.from_dict(clone_snippets_with_vuln, orient='index')
else:
    print("Check next repository")
    shutil.rmtree('to_check_functions-blind-crossclones')
    all_files = os.listdir()

    for item in all_files:
        if item.endswith(".log"):
            os.remove(item)

    for item in all_files:
        if item.startswith(folder) and item.endswith('.xml'):
            os.remove(item)
    os.system('rmdir /S /Q "{}"'.format(folder))

In [ ]:
clone_snippets_with_vuln_df = clone_snippets_with_vuln_df.drop_duplicates().reset_index(drop=True)
clone_snippets_with_vuln_df

In [ ]:
ghCodeSnippetHistory = pd.DataFrame()

In [ ]:
os.system('rmdir /S /Q "{}"'.format(folder))
repo = git.Repo.clone_from('https://github.com/'+repo_name+'.git', folder)

In [ ]:
chdir(ROOT_PATH+'/'+folder)
getcwd()

In [ ]:
proc = subprocess.run(["git",  
                   "log",
                   '-M',
                   '--diff-filter=R',
                   "--summary",
                   "--reverse",
                      '>',
                      'file_renamed.txt'],
                  shell=True) # if at the latest commit the path does not exist

In [ ]:
 # if at the latest commit the path does not exist
index_row = 0
with open('file_renamed.txt', 'r') as f:
    print(clone_snippets_with_vuln_df['file'][index_row].split('/')[-1])
    lines = f.readlines()
    replaced = False
    for line in lines:      
            if clone_snippets_with_vuln_df['file'][index_row].split('/')[-1] in line:
                print(line)
                sx_bracket = line.index('{')
                dx_bracket = line.index('}')
                index_arrow = line.index('=>')

                print("Old")
                old = line[sx_bracket+1:index_arrow-1]
                print(old.strip())
                print("To substitute")
                substitute = line[index_arrow+3:dx_bracket]
                print(substitute.strip())
                old_path = clone_snippets_with_vuln_df['file'][index_row]
                clone_snippets_with_vuln_df['file'][index_row] = clone_snippets_with_vuln_df['file'][index_row].replace(old.strip(), substitute.strip())
                replaced=True
                print(clone_snippets_with_vuln_df['file'][index_row].split('/')[-1])
if replaced == False:
    print("The file may be deleted")
else:
    print("The file path is renamed")

In [ ]:
for index in range(clone_snippets_with_vuln_df.shape[0]):
    print('Key_index '+str(index))
    print(clone_snippets_with_vuln_df['file'][index])
    repo.git.checkout('master')
    proc = subprocess.run(["git", 
                           "log",
                           '--pretty=format:%H',
                           "--follow",
                           "--",
                           clone_snippets_with_vuln_df['file'][index]], 
                          capture_output=True) 

    commits = proc.stdout.decode('utf-8').split('\n')
    commits.reverse()

        
    start_from = commits.index(clone_snippets_with_vuln_df['origin_commit'][index])+1

    print("Start from "+str(start_from))
    
    if len(commits)==1:
        print("This is the only commit on this file")
    else:
        try:
            for number in range(start_from, len(commits)):
                repo.git.checkout(commits[number])
                print('Checkout at commit '+commits[number])
                to_delete = []
                add_to_code_context = []
                
                print(commits[number])
                print("Run diff for "+clone_snippets_with_vuln_df['file'][index])
                proc = subprocess.run(["git", 
                                       "diff",
                                       commits[number-1]+".."+commits[number],
                                       '--',
                                        clone_snippets_with_vuln_df['file'][index]
                                      ],
                                      capture_output=True)

                if proc.stdout:
                    deleted = []
                    added = []
                    print("Diff for "+clone_snippets_with_vuln_df['file'][index])
                    with open('diff.txt', 'w', encoding='utf-8') as f:
                        f.write(proc.stdout.decode('utf-8'))
                    with open('diff.txt', 'r') as f:
                        context_index = None
                        context_line = None
                        print("Read diff")
                        content = f.readlines()

                        for line in content:
                            if line.startswith('-'):
                                deleted.append(line[1:])
                            if line.startswith('+'):
                                added.append(line[1:])

                        print("Analyze lines added")
                        print(added)
                        for added_line in added:
                            added_line_index = None
                            
                            if not added_line.startswith('++'):
                                print("Check "+added_line)
                
                                with open(clone_snippets_with_vuln_df['file'][index], 'r') as f:
                                    lines_cc = f.readlines();


                                    for x in lines_cc:
                                        x = x.replace('\n', '').replace('\r', '')
                                        if x != '' and x in clone_snippets_with_vuln_df['code_context'][index].split('\r\n')[0]:
                                            context_line = x
                                            print(x)
                                            print(clone_snippets_with_vuln_df['code_context'][index])
                                            print('###')
                                            break;

                                    print(context_index)
                                    print(added_line)
                                    if context_line:
                                        context_index = lines_cc.index(context_line+'\n')
                                        added_line_index = lines_cc.index(added_line)

                                        print(context_index)
                                        print(added_line_index)
                                        row_diff = context_index - added_line_index
                                        row_diff = abs(row_diff)
                                        print(row_diff)
                                        if row_diff>=0 and row_diff<=3:
                                            print("Add to code context")
                                            add_to_code_context.append(added_line)

                        print("Analyze lines deleted")
                        for line_deleted in deleted:
                            line_deleted = line_deleted.replace('\n', '')
                            if line_deleted:
                                if line_deleted  in clone_snippets_with_vuln_df['code_context'][index]:
                                    if line_deleted and line_deleted != '{' and line_deleted != '}':
                                        print("Append row")

                                        to_delete.append(line_deleted)



                to_change =  clone_snippets_with_vuln_df['code_context'][index]
                print("Code context updated before")
                print(to_change)
                if to_delete:
                    for entry in to_delete:
                        to_change = to_change.replace(entry, "")

                    print(commits[number])
                    print(to_delete)
                     #se sono rmaste solo parentesi graffe allora vedi se il file  stato spostato

                    #analyze_moves = False
                    for line in to_change:
                        if line == '\r' or line == '\n' or line == '{' or line=='}':
                            to_change = to_change.replace(line, '')
                if add_to_code_context:
                    string_to_add = ' '.join(add_to_code_context)
                    to_change = string_to_add + to_change

                print("Code context updated after")
                print(to_change)

                if to_change == '':
                    print('Check moves for '+clone_snippets_with_vuln_df['file'][index][1:])
                
                with Popen(
                    args=['git', 'show', '-s', '--format=%ci' ,  commits[number]],
                    shell=False,
                    stdout=PIPE,
                    bufsize=1,
                    universal_newlines=True,
                ) as process:

                    for line in process.stdout:
                        date = line.strip()
                ghCodeSnippetHistory = ghCodeSnippetHistory.append({'repo': repo_name, 'origin_commit': commits[number], 
                                                    'file': clone_snippets_with_vuln_df['file'][index], 
                                           'code_context': to_change, 
                                                    'commit_date': date}, ignore_index=True)
                print()
            
        except UnicodeDecodeError:
            print("Encoding error")

In [ ]:
chdir(ROOT_PATH)
getcwd()

In [ ]:
ghCodeSnippetHistory

In [ ]:
rows_to_check_move = ghCodeSnippetHistory.loc[ghCodeSnippetHistory['code_context']=='']    

In [ ]:
rows_to_check_move

In [ ]:
chdir(ROOT_PATH+'/'+folder)

In [ ]:
#skip if rows_to_check_move is empty
repo.git.checkout('master')
proc = subprocess.run(["git", 
                   "log",
                   '-M',
                   '--diff-filter=R',
                   "--summary",
                   "--reverse",
                      '>',
                      'file_renamed.txt'],
                  shell=True) 

In [ ]:
# skip if rows_to_check_move is empty
files = list(pd.unique(rows_to_check_move['file']))
print(len(files))
for _, row in rows_to_check_move.iterrows():
    lines_collected = []
    start_from_this_commit = row['origin_commit']
    start_read = False 
    replaced = False

    with open('file_renamed.txt', 'r') as f:
        
        lines = f.readlines()
        show = False
        for line in lines:
            if line.strip().startswith('commit '+start_from_this_commit):
                show = True
            
            if line.strip().startswith('commit ') and not line.strip().startswith('commit '+start_from_this_commit):
                show = False
            if show:
                if row['file'].split('/')[-1] in line and row['file'] in files:
                    print(line)
                    start = line.index('{')+1
                    end = line.index('}')
                    between_brackets = line[start:end]
                    index_arrow = between_brackets.index('=>')
                    print(between_brackets)
                    print("Old")
                    old = between_brackets[0:index_arrow]
                    print(old.strip())
                    print("To substitute")
                    substitute = between_brackets[index_arrow+3:len(between_brackets)]
                    print(substitute.strip())
                    old_path = row['file']
                    files.remove(old_path)
                    row['file'] = row['file'].replace(old.strip(), substitute.strip())
                    replaced = True
                    print(row['file'])
                    
                    with Popen(
                            args=['git', 'show', '-s', '--format=%ci' ,  start_from_this_commit],
                            shell=False,
                            stdout=PIPE,
                            bufsize=1,
                            universal_newlines=True,
                        ) as process:

                            for line in process.stdout:
                                date = line.strip()
                    index_row =  clone_snippets_with_vuln_df.loc[clone_snippets_with_vuln_df['file']==old_path].index[0]
                    ghCodeSnippetHistory.loc[_] = {'repo': repo_name, 'origin_commit': start_from_this_commit, 
                                    'file': row['file'], 
                           'code_context': clone_snippets_with_vuln_df.iloc[index_row, 3]    , 
                                    'commit_date': date}
                    

    if replaced == False and row['file'] in files:
        print("The file may be deleted")
        ghCodeSnippetHistory = ghCodeSnippetHistory.drop([_])
    if replaced==True:
        print("The file path is renamed")

In [ ]:
chdir(ROOT_PATH)
getcwd()

In [ ]:
ghCodeSnippetHistory

In [ ]:
chdir(ROOT_PATH+'/'+folder)
getcwd()

In [ ]:
len(ghCodeSnippetHistory)

In [ ]:
ghCodeSnippetHistory = ghCodeSnippetHistory.reset_index(drop = True)

In [ ]:
ghCodeSnippetHistory

In [ ]:
ghCodeSnippetHistory_aftercppcheck = dict()
idx = 0
num = 0
for _, row in ghCodeSnippetHistory.iterrows():
    if _ == 1:
        path = 'xbmc/cores/paplayer/NSFCodec/src/memguard.c'
    else:
        path = row['file']
    print(_)
    print(row['origin_commit'])
    print(path)
    repo.git.checkout(row['origin_commit'])
    proc = subprocess.run(["cppcheck", "--enable=style", "--template={cwe};{file};{line};{severity};{code}", path], capture_output=True)
    result = proc.stderr.decode('utf-8')
    print(proc.stdout)

    if proc.stdout.decode('utf-8') == 'cppcheck: error: could not find or open any of the paths given.\r\n':
        continue;

    if result:
        items = result.split('\r\n')
        items = list(filter(None, items))
        for x, y in zip(*[iter(items)] * 2):
            row = x + y
            row = row.split(';')

            if not row[2]=='0':
                print("Flaw found at line "+row[2])
                ghCodeSnippetHistory_aftercppcheck[idx] = {
                    'repo': ghCodeSnippetHistory['repo'][_],
                    'origin_commit': ghCodeSnippetHistory['origin_commit'][_],
                    'file': ghCodeSnippetHistory['file'][_],
                    'code_context': ghCodeSnippetHistory['code_context'][_],
                    'commit_date': ghCodeSnippetHistory['commit_date'][_],
                    'line_vuln': row[2],
                    'code_vuln': row[4]
                }
                idx = idx + 1
            else:
                print("Syntax error")
                break;
    else:
        ghCodeSnippetHistory_aftercppcheck[idx] = {
            'repo': ghCodeSnippetHistory['repo'][_],
            'origin_commit': ghCodeSnippetHistory['origin_commit'][_] ,
            'file': ghCodeSnippetHistory['file'][_] ,
            'code_context': ghCodeSnippetHistory['code_context'][_], 
            'commit_date': ghCodeSnippetHistory['commit_date'][_],
            'line_vuln': '',
            'code_vuln': ''
        }
        idx = idx + 1
    num = num + 1
    print() 

In [ ]:
ghCodeSnippetHistory_aftercppcheck

In [ ]:
ghCodeSnippetHistory_aftercppcheck = pd.DataFrame.from_dict(ghCodeSnippetHistory_aftercppcheck, orient='index')

In [ ]:
ghCodeSnippetHistory_aftercppcheck = ghCodeSnippetHistory_aftercppcheck.reset_index(drop=True)

In [ ]:
len(ghCodeSnippetHistory_aftercppcheck)

In [ ]:
ghCodeSnippetHistory_aftercppcheck

In [ ]:
files = pd.unique(ghCodeSnippetHistory_aftercppcheck['file'])

for file in files:
    print(file)

In [ ]:
clone_snippets_with_vuln_df.loc[clone_snippets_with_vuln_df['file']==files[0]]

In [ ]:
ghCodeSnippetHistory_aftercppcheck.loc[ghCodeSnippetHistory_aftercppcheck['file']==files[0]]

In [ ]:
code_initial = pd.unique(clone_snippets_with_vuln_df['code_vuln'])
indexes = []
for code in code_initial:
    code = code.replace('\r', '').replace('\n','').replace('^','') 
    

    for _,row in ghCodeSnippetHistory_aftercppcheck.iterrows():
        code_gh = ghCodeSnippetHistory_aftercppcheck['code_vuln'][_]
        code_gh = code_gh.replace('\r', '').replace('\n','').replace('^','') 
        if code in code_gh:
            indexes.append(_)

In [ ]:
ghCodeSnippetHistory_aftercppcheck.iloc[indexes].drop_duplicates()# where code_vuln is equal

In [ ]:
ghCodeSnippetHistory_aftercppcheck

In [ ]:
chdir(ROOT_PATH)
ghCodeSnippetHistory_aftercppcheck.to_csv('GHCodeSnippetHistory/history_'+file_name+".csv")